In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_160x36x36_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_160x36x36_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_tscan(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_tscan(train).cache(f'{tmp}/tscan'), to_tf_tscan(valid).cache(f'{tmp}/tscan')
train, valid = to_tf_tscan(train), to_tf_tscan(valid)

(160, 36, 36, 3)


In [16]:
ts_can_e2e = TS_CAN_end_to_end(n=20)
ts_can_e2e.compile(optimizer='Nadam', loss='mse')
ts_can_e2e.build(input_shape=(None, 36, 36, 3))
print(f'Flops per frame:{get_flops(ts_can_e2e, input_sig=[tf.TensorSpec([160, 36, 36, 3])])/160:.0f}')
ts_can_e2e.summary()

Flops per frame:52158718
Model: "ts_can_end_to_end_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_5 (Functional)         (None, 1)                 532899    
Total params: 532,899
Trainable params: 532,899
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ts_can_e2e.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/TS-CAN_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
ts_can_e2e.load_weights('../weights/TS-CAN_CCNU.h5')

In [17]:
ts_can_e2e.load_weights('../weights/TS-CAN_CCNU.h5')

In [4]:
def cumsum(result, overwrite=True):
    with h5py.File(result, 'a') as f:
        for i, j in f.items():
            if 'diff' not in j.keys():
                j.create_dataset('diff', data=j['predict'])
                j['predict'][...] = detrend(np.cumsum(j['diff']))
            elif overwrite:
                j['predict'][...] = detrend(np.cumsum(j['diff']))

In [11]:
eval_on_dataset(test_set_CCNU, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
cumsum('../results/TS-CAN_CCNU_CCNU.h5')
get_metrics('../results/TS-CAN_CCNU_CCNU.h5')

100%|██████████| 179/179 [09:46<00:00,  3.28s/it]


{'Sliding window': {'MAE': 1.223, 'RMSE': 3.526, 'R': 0.93915},
 'Whole video': {'MAE': 0.929, 'RMSE': 2.898, 'R': 0.95286}}

In [4]:
#eval_on_dataset(test_set_CCNU_rPPG, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
#cumsum('../results/TS-CAN_CCNU_CCNU_rPPG.h5')
#get_metrics('../results/TS-CAN_CCNU_CCNU_rPPG.h5')
get_metrics_HRV('../results/TS-CAN_CCNU_CCNU_rPPG.h5')

{'SDNN': {'MAE': 50.056, 'RMSE': 59.304, 'R': 0.39467}}

In [3]:
#eval_on_dataset(test_set_PURE, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
#cumsum('../results/TS-CAN_CCNU_PURE.h5')
#get_metrics('../results/TS-CAN_CCNU_PURE.h5')
get_metrics_HRV('../results/TS-CAN_CCNU_PURE.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


{'SDNN': {'MAE': 61.401, 'RMSE': 74.103, 'R': 0.29331}}

In [2]:
#eval_on_dataset(test_set_UBFC_rPPG2, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
#cumsum('../results/TS-CAN_CCNU_UBFC.h5')
#get_metrics('../results/TS-CAN_CCNU_UBFC.h5')
get_metrics_HRV('../results/TS-CAN_CCNU_UBFC.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\scipy\interpolate\_fitpack2.py:298: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


{'SDNN': {'MAE': 25.607, 'RMSE': 31.848, 'R': 0.5878}}

In [26]:
#eval_on_dataset(test_set_MMPD, ts_can_e2e, 160, (36, 36), step=1, batch=0, save='../results/TS-CAN_CCNU_MMPD.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/TS-CAN_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'LED-low', 'Incandescent'])

{'Sliding window': {'MAE': 1.715, 'RMSE': 4.864, 'R': 0.85781},
 'Whole video': {'MAE': 0.95, 'RMSE': 1.396, 'R': 0.98897}}